# DataFrames in pandas
A set of examples that exhibit some of the core features of the [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) data type in the `pandas` module.

In [240]:
import numpy as np
import pandas as pd

## Basic concept
A DataFrame is a two-dimensional tabular data struture.  It is easily visualized like a spreadsheet, with rows and columns.

In [241]:
# create a DataFrame from a dictionary containing labeled pandas Series
df = pd.DataFrame({
    'name': pd.Series( ['Foo', 'Bar', 'Baz'] ),
    'email': pd.Series( ['fo1258@foo.edu', 'br9876@foo.edu', 'bz2292@foo.edu'] ),
    'midterm exam': pd.Series( [99, 64, 87] ),
    'final exam': pd.Series( [94, 72, 81] )
})
df

name           email  midterm exam  final exam
0  Foo  fo1258@foo.edu            99          94
1  Bar  br9876@foo.edu            64          72
2  Baz  bz2292@foo.edu            87          81

In [242]:
# get the DataFrame's schema
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          3 non-null      object
 1   email         3 non-null      object
 2   midterm exam  3 non-null      int64 
 3   final exam    3 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 224.0+ bytes


### Columns as Series
Each column is a named `pandas` Series.

In [243]:
df['midterm exam']

0    99
1    64
2    87
Name: midterm exam, dtype: int64

In [244]:
# prove that a column of a DataFrame is a Series
type( df['midterm exam'] )

pandas.core.series.Series

## Rows
Each row is also considered a `pandas` Series.

In [245]:
# get a row by its index
df.loc[1]

name                       Bar
email           br9876@foo.edu
midterm exam                64
final exam                  72
Name: 1, dtype: object

In [246]:
# prove that a row of a DataFrame is a Series
type( df.loc[1] )

pandas.core.series.Series

In [247]:
# get a row by its integer index
df.iloc[2]

name                       Baz
email           bz2292@foo.edu
midterm exam                87
final exam                  81
Name: 2, dtype: object

## Filtering rows


In [248]:
# match a criterion
df[ df['name'] == 'Bar' ]

name           email  midterm exam  final exam
1  Bar  br9876@foo.edu            64          72

In [249]:
# match multiple criteria using & or | logic operators
df[ (df['name'] != 'Bar') & (df['midterm exam'] > 50) ]

name           email  midterm exam  final exam
0  Foo  fo1258@foo.edu            99          94
2  Baz  bz2292@foo.edu            87          81

## Filtering columns

Extracting a **single column** is straightforward with square bracket syntax.

In [250]:
# fetch the 'name' column - this returns a Series
df['name']

0    Foo
1    Bar
2    Baz
Name: name, dtype: object

The easiest way to extract **multiple columns** from a dataframe is by supplying a list of column names.

In [251]:
# fetch the 'name' and 'final exam' columns - this returns a DataFrame
df[ ['name', 'final exam'] ]

name  final exam
0  Foo          94
1  Bar          72
2  Baz          81

## Filtering rows and columns

It is possible to use two sets of brackets to perform both row and column filters in one expression.

In [252]:
# find one row by its index, and fetch one column from the results - this returns a single value
df.loc[2]['final exam']

81

In [253]:
# filter rows by criteria, and fetch one column from the results - this returns a Series
df[ df['name'] != 'Baz']['midterm exam']

0    99
1    64
Name: midterm exam, dtype: int64

In [254]:
# filter rows, and fetch multiple columns from the results - this returns a DataFrame
df[ df['name'] != 'Baz'][ ['name', 'midterm exam'] ] 

name  midterm exam
0  Foo            99
1  Bar            64

## Basic operations

In [255]:
# give a flat 2% curve to all students on the midterm exam
# update the midterm exam column
df['midterm exam'] = df['midterm exam'] + 2
df

name           email  midterm exam  final exam
0  Foo  fo1258@foo.edu           101          94
1  Bar  br9876@foo.edu            66          72
2  Baz  bz2292@foo.edu            89          81

In [256]:
# add a new column to the dataframe...

# first, generate a Series of fake student ids
n_numbers = pd.Series(100000000*np.random.random(3)) # generate a series of random numbers
n_numbers = n_numbers.astype(int) # convert to a simple int to remove decimal place
n_numbers = 'N' + n_numbers.map(str) # add the letter 'N' in front of each number (first convert each to str)

# add to dataframe as a new column
df['n number'] = n_numbers
df

name           email  midterm exam  final exam   n number
0  Foo  fo1258@foo.edu           101          94  N63047275
1  Bar  br9876@foo.edu            66          72  N60309747
2  Baz  bz2292@foo.edu            89          81  N99581936

## Mergine two dataframes

In [257]:
# let's first create a second dataframe with some more information about each student
# note that one of the indices in this dataframe does not exist in the other dataframe
df2 = pd.DataFrame({
    'major': ['Math', 'Computer Science', 'Philosophy', 'Organic Gardening'],
    'minor': ['Art History', 'Linguistics', 'Music Performance', 'Theater Lighting']
}, index = [3, 0, 2, 1])

df2

major              minor
3               Math        Art History
0   Computer Science        Linguistics
2         Philosophy  Music Performance
1  Organic Gardening   Theater Lighting

In [258]:
# do an "inner join" type merge, where referential integrity is maintained
df.join(df2)

name           email  midterm exam  final exam   n number  \
0  Foo  fo1258@foo.edu           101          94  N63047275   
1  Bar  br9876@foo.edu            66          72  N60309747   
2  Baz  bz2292@foo.edu            89          81  N99581936   

               major              minor  
0   Computer Science        Linguistics  
1  Organic Gardening   Theater Lighting  
2         Philosophy  Music Performance

In [259]:
# do a "left join" type merge, where referential integrity is not maintained
pd.concat( [df, df2], axis=1)

name           email  midterm exam  final exam   n number  \
0  Foo  fo1258@foo.edu         101.0        94.0  N63047275   
1  Bar  br9876@foo.edu          66.0        72.0  N60309747   
2  Baz  bz2292@foo.edu          89.0        81.0  N99581936   
3  NaN             NaN           NaN         NaN        NaN   

               major              minor  
0   Computer Science        Linguistics  
1  Organic Gardening   Theater Lighting  
2         Philosophy  Music Performance  
3               Math        Art History

## Setting the index
It's possible to change which column is used as an index

In [260]:
# set the index to be the new n number
df.set_index('n number')

name           email  midterm exam  final exam
n number                                                
N63047275  Foo  fo1258@foo.edu           101          94
N60309747  Bar  br9876@foo.edu            66          72
N99581936  Baz  bz2292@foo.edu            89          81

## Importing data from files
Pandas can import from a variety of common data file formats, including CSV, JSON, fixed-width column text, and more.

In [261]:
# open data about NYC jobs from https://data.cityofnewyork.us/City-Government/NYC-Jobs/kpav-sd4t
df = pd.read_csv('./NYC_Jobs.csv')

In [262]:
# get the DataFrame's schema - notice the auto-detection of data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Job ID                         1218 non-null   int64  
 1   Agency                         1218 non-null   object 
 2   Posting Type                   1218 non-null   object 
 3   # Of Positions                 1218 non-null   int64  
 4   Business Title                 1218 non-null   object 
 5   Civil Service Title            1218 non-null   object 
 6   Title Classification           1218 non-null   object 
 7   Title Code No                  1218 non-null   object 
 8   Level                          1218 non-null   object 
 9   Job Category                   1216 non-null   object 
 10  Full-Time/Part-Time indicator  1129 non-null   object 
 11  Career Level                   1216 non-null   object 
 12  Salary Range From              1218 non-null   f

In [263]:
# show a few randomly-sampled rows
df.sample(5)

Job ID                         Agency Posting Type  # Of Positions  \
245  452572  DEPT OF HEALTH/MENTAL HYGIENE     External               1   
232  416466          NYC HOUSING AUTHORITY     Internal               5   
138  442016     FINANCIAL INFO SVCS AGENCY     External               1   
219  431033        HUMAN RIGHTS COMMISSION     Internal               1   
198  458424               CONSUMER AFFAIRS     Internal               1   

                                        Business Title  \
245  Associate Laboratory Microbiologist, Bureau of...   
232                                    Field Inspector   
138                          DISASTER RECOVERY ANALYST   
219  Program & Policy Specialist for Racial and Res...   
198                                    Press Secretary   

                Civil Service Title Title Classification Title Code No Level  \
245  ASSOCIATE LABORATORY MICROBIOL        Competitive-1         21514    01   
232    CONSTRUCTION PROJECT MANAGER        Competitive-1         34202    01   
138  IT SERVICE MANAGEMENT SPECIALI    Non-Competitive-5         95713    00   
219  ASSOCIATE HUMAN RIGHTS SPECIAL        Competitive-1         55038    01   
198  ADMINISTRATIVE PUBLIC INFORMAT        Competitive-1         10033    M2   

                                   Job Category  ...  \
245                                      Health  ...   
232       Engineering, Architecture, & Planning  ...   
138               Technology, Data & Innovation  ...   
219   Constituent Services & Community Programs  ...   
198  Communications & Intergovernmental Affairs  ...   

                                Additional Information  \
245  **Applicant must possess or be eligible for a ...   
232  1.  Employees serving in the titles of or who ...   
138                                               P461   
219                                                NaN   
198                                                NaN   

                                              To Apply  \
245  Apply online with a cover letter to https://a1...   
232                        Click the Apply Now button.   
138  External applicants please visit https://a127-...   
219  For City employees: Go to Employee Self-Servic...   
198  For Non-City/External Candidates: Visit the Ex...   

                                           Hours/Shift  Work Location 1  \
245                                                NaN              NaN   
232                                                NaN              NaN   
138                     Monday â Friday, 9am to 5pm.              NaN   
219  9AM-5PM; ON OCCASION, CANDIDATES WILL BE REQUI...              NaN   
198                                                NaN              NaN   

    Recruitment Contact                              Residency Requirement  \
245                 NaN  New York City residency is generally required ...   
232                 NaN               NYCHA has no residency requirements.   
138                 NaN  New York City Residency is not required for th...   
219                 NaN  New York City residency is generally required ...   
198                 NaN  New York City residency is generally required ...   

    Posting Date   Post Until Posting Updated Process Date  
245   01/29/2021  29-MAY-2021      01/29/2021   04/13/2021  
232   03/26/2021          NaN      03/26/2021   04/13/2021  
138   07/23/2020          NaN      07/23/2020   04/13/2021  
219   01/29/2020          NaN      01/29/2020   04/13/2021  
198   02/23/2021  24-APR-2021      02/23/2021   04/13/2021  

[5 rows x 30 columns]

In [264]:
# look for good-paying jobs ( > $200,000) available for external candidates
df_external_jobs = df[ df['Posting Type'] == 'External' ]
df_external_annual_jobs = df_external_jobs[ df_external_jobs['Salary Frequency'] == 'Annual' ]
df_external_annual_jobs_over_200k = df_external_annual_jobs[ df_external_annual_jobs['Salary Range To'] >= 200000 ]
df_external_annual_jobs_over_200k.sample(5)

Job ID                         Agency Posting Type  # Of Positions  \
1052  459311  OFFICE OF MANAGEMENT & BUDGET     External               1   
176   432041      ADMIN FOR CHILDREN'S SVCS     External               1   
438   458110      OFFICE OF THE COMPTROLLER     External               1   
644   453677   NYC EMPLOYEES RETIREMENT SYS     External               1   
773   457622   DEPT OF INFO TECH & TELECOMM     External               1   

                                        Business Title  \
1052     Assistant Director  Administration of Justice   
176   Deputy Commissioner, Child and Family Well-Being   
438                                 Chief Risk Officer   
644                  ADMINISTRATIVE MANAGEMENT AUDITOR   
773                     Deputy CISO - Urban Technology   

                 Civil Service Title      Title Classification Title Code No  \
1052  BUDGET ANALYST (OMB)-MANAGERIA  Pending Classification-2         0608A   
176   DEPUTY DIRECTOR OF ADMINISTRAT         Non-Competitive-5         52485   
438   DIRECTOR OF  INVESTMENTS (COMP         Non-Competitive-5         95612   
644   ADMINISTRATIVE MANAGEMENT AUDI             Competitive-1         10010   
773        TELECOMMUNICATION MANAGER             Competitive-1         82984   

     Level                                       Job Category  ...  \
1052    M4  Finance, Accounting, & Procurement Policy, Res...  ...   
176     M7                                    Social Services  ...   
438     MY                 Finance, Accounting, & Procurement  ...   
644     M5                   Administration & Human Resources  ...   
773     M5                      Technology, Data & Innovation  ...   

                                 Additional Information  \
1052  REQUIREMENTS:  Assistant Director ($141,766): ...   
176   Section 424-A of the New York Social Services ...   
438   The selected candidate will be subject to the ...   
644                                                 NaN   
773                                                 NaN   

                                               To Apply  \
1052  For City employees, please go to Employee Self...   
176                           Click on Apply Now button   
438   Please visit our website at https://comptrolle...   
644   TO APPLY FOR CONSIDERATION, PLEASE FORWARD A C...   
773   Special Note: Taking and passing civil service...   

                                            Hours/Shift       Work Location 1  \
1052                                                NaN  255 Greenwich Street   
176                                                 NaN                   NaN   
438                                                 NaN                   NaN   
644                                                 NaN                   NaN   
773   Day - Due to the necessary technical managemen...          New York, NY   

     Recruitment Contact                              Residency Requirement  \
1052                 NaN  New York City residency is generally required ...   
176                  NaN  New York City residency is generally required ...   
438                  NaN  New York City residency is generally required ...   
644                  NaN  New York City residency is generally required ...   
773                  NaN  New York City residency is generally required ...   

     Posting Date Post Until Posting Updated Process Date  
1052   03/10/2021        NaN      03/10/2021   04/13/2021  
176    02/04/2020        NaN      02/04/2020   04/13/2021  
438    02/08/2021        NaN      02/08/2021   04/13/2021  
644    11/17/2020        NaN      11/17/2020   04/13/2021  
773    02/03/2021        NaN      02/11/2021   04/13/2021  

[5 rows x 30 columns]

In [265]:
# the same query as above, just in one line
df[ (df['Posting Type'] == 'External') & (df['Salary Frequency'] == 'Annual') & (df['Salary Range To'] >= 200000) ].sample(5)

Job ID                        Agency Posting Type  # Of Positions  \
401  459041  NYC EMPLOYEES RETIREMENT SYS     External               1   
759  373748  NYC EMPLOYEES RETIREMENT SYS     External               1   
263  445754         NYC HOUSING AUTHORITY     External               1   
734  441706   TAXI & LIMOUSINE COMMISSION     External               1   
332  434222  NYC EMPLOYEES RETIREMENT SYS     External               1   

                                        Business Title  \
401                       ADMINISTRATIVE STAFF ANALYST   
759                  ADMINISTRATIVE MANAGEMENT AUDITOR   
263      Vice President for Operation Support Services   
734  General Counsel/Deputy Commissioner for Legal ...   
332                           COMPUTER SYSTEMS MANAGER   

                Civil Service Title Title Classification Title Code No Level  \
401  ADMINISTRATIVE STAFF ANALYST (        Competitive-1         10026    M7   
759  ADMINISTRATIVE MANAGEMENT AUDI        Competitive-1         10010    M7   
263  ADMINISTRATIVE HOUSING SUPERIN        Competitive-1         10019    M5   
734        EXECUTIVE AGENCY COUNSEL    Non-Competitive-5         95005    M6   
332        COMPUTER SYSTEMS MANAGER        Competitive-1         10050    M7   

                                          Job Category  ...  \
401          Public Safety, Inspections, & Enforcement  ...   
759  Administration & Human Resources Finance, Acco...  ...   
263                  Building Operations & Maintenance  ...   
734                                      Legal Affairs  ...   
332                      Technology, Data & Innovation  ...   

                                Additional Information  \
401                                                NaN   
759                                                NaN   
263  1.\tNYCHA employees applying for promotional, ...   
734                                                NaN   
332                                                NaN   

                                              To Apply  Hours/Shift  \
401  TO APPLY FOR CONSIDERATION, PLEASE FORWARD A C...          NaN   
759  TO APPLY FOR CONSIDERATION, PLEASE FORWARD A C...          NaN   
263                        Click the Apply Now button.          NaN   
734  Click, APPLY NOW Current city employees must a...          NaN   
332  TO APPLY FOR CONSIDERATION, PLEASE FORWARD A C...          NaN   

               Work Location 1 Recruitment Contact  \
401                        NaN                 NaN   
759                        NaN                 NaN   
263                        NaN                 NaN   
734  33 Beaver St, New York Ny                 NaN   
332                        NaN                 NaN   

                                 Residency Requirement Posting Date  \
401  New York City residency is generally required ...   03/03/2021   
759  New York City residency is generally required ...   11/07/2018   
263               NYCHA has no residency requirements.   09/23/2020   
734  New York City residency is generally required ...   07/13/2020   
332  New York City Residency is not required for th...   02/18/2020   

      Post Until Posting Updated Process Date  
401  30-APR-2021      03/03/2021   04/13/2021  
759          NaN      11/07/2018   04/13/2021  
263          NaN      02/16/2021   04/13/2021  
734          NaN      07/13/2020   04/13/2021  
332          NaN      02/18/2020   04/13/2021  

[5 rows x 30 columns]

## Basic statistics

In [266]:
# get an overview of most common stats
df.describe()

Job ID  # Of Positions  Salary Range From  Salary Range To  \
count    1218.000000     1218.000000        1218.000000      1218.000000   
mean   422305.477011        4.671593       54910.929420     80338.174449   
std     69258.267489       18.947981       33388.197579     52242.387389   
min     87990.000000        1.000000           0.000000        10.360000   
25%    426293.750000        1.000000       38333.000000     55123.000000   
50%    456028.000000        1.000000       58700.000000     78303.000000   
75%    460054.500000        1.000000       74650.000000    108903.750000   
max    461151.000000      250.000000      250000.000000    265000.000000   

       Recruitment Contact  
count                  0.0  
mean                   NaN  
std                    NaN  
min                    NaN  
25%                    NaN  
50%                    NaN  
75%                    NaN  
max                    NaN

In [267]:
# the same, but just for the 'Salary Range To' field (which is a Series, of course)
df['Salary Range To'].describe()

count      1218.000000
mean      80338.174449
std       52242.387389
min          10.360000
25%       55123.000000
50%       78303.000000
75%      108903.750000
max      265000.000000
Name: Salary Range To, dtype: float64

In [268]:
# get just the mean from the column
df['Salary Range To'].median()

78303.0

The other statistics functions - `min()`, `max()`, `mean()`, `std()`, `count()` - work similarly.

## Grouping by a column

In [271]:
# count how many jobs are in each agency
df.groupby("Agency")['Agency'].count()

Agency
ADMIN FOR CHILDREN'S SVCS          55
ADMIN TRIALS AND HEARINGS           4
BOARD OF CORRECTION                 2
BOROUGH PRESIDENT-QUEENS            2
BUSINESS INTEGRITY COMMISSION       7
CIVILIAN COMPLAINT REVIEW BD        2
CONSUMER AFFAIRS                   34
DEPARTMENT FOR THE AGING           16
DEPARTMENT OF BUILDINGS             3
DEPARTMENT OF BUSINESS SERV.        4
DEPARTMENT OF CITY PLANNING        31
DEPARTMENT OF CORRECTION           51
DEPARTMENT OF INVESTIGATION        18
DEPARTMENT OF PROBATION             6
DEPARTMENT OF SANITATION            8
DEPARTMENT OF TRANSPORTATION       12
DEPT OF CITYWIDE ADMIN SVCS         9
DEPT OF ENVIRONMENT PROTECTION    123
DEPT OF HEALTH/MENTAL HYGIENE      85
DEPT OF INFO TECH & TELECOMM       53
DEPT OF PARKS & RECREATION         46
DEPT OF YOUTH & COMM DEV SRVS      18
DISTRICT ATTORNEY KINGS COUNTY      4
DISTRICT ATTORNEY RICHMOND COU      4
FINANCIAL INFO SVCS AGENCY         36
FIRE DEPARTMENT                    14
HOUSI

In [278]:
# calculate the mean top salary within each agency
df.groupby("Agency")['Salary Range To'].mean()

Agency
ADMIN FOR CHILDREN'S SVCS          96378.202182
ADMIN TRIALS AND HEARINGS          18905.576350
BOARD OF CORRECTION               150000.000000
BOROUGH PRESIDENT-QUEENS           63794.000000
BUSINESS INTEGRITY COMMISSION      87857.142857
CIVILIAN COMPLAINT REVIEW BD      108064.500000
CONSUMER AFFAIRS                   69002.576471
DEPARTMENT FOR THE AGING           79165.859337
DEPARTMENT OF BUILDINGS            74076.666667
DEPARTMENT OF BUSINESS SERV.      137641.750000
DEPARTMENT OF CITY PLANNING        77604.548387
DEPARTMENT OF CORRECTION           76070.957784
DEPARTMENT OF INVESTIGATION        76121.888889
DEPARTMENT OF PROBATION            62702.333333
DEPARTMENT OF SANITATION          143626.850000
DEPARTMENT OF TRANSPORTATION      115114.530000
DEPT OF CITYWIDE ADMIN SVCS        57031.733333
DEPT OF ENVIRONMENT PROTECTION     31538.537340
DEPT OF HEALTH/MENTAL HYGIENE      70782.981556
DEPT OF INFO TECH & TELECOMM      145506.735849
DEPT OF PARKS & RECREATION       

In [282]:
# show just the top 10 paying agencies
df.groupby("Agency")['Salary Range To'].mean().sort_values().tail(10)

Agency
TAXI & LIMOUSINE COMMISSION       103237.086957
CIVILIAN COMPLAINT REVIEW BD      108064.500000
DEPARTMENT OF TRANSPORTATION      115114.530000
MAYORS OFFICE OF CONTRACT SVCS    118550.000000
NYC EMPLOYEES RETIREMENT SYS      131359.198963
DEPARTMENT OF BUSINESS SERV.      137641.750000
DEPARTMENT OF SANITATION          143626.850000
DEPT OF INFO TECH & TELECOMM      145506.735849
BOARD OF CORRECTION               150000.000000
NYC FIRE PENSION FUND             156076.000000
Name: Salary Range To, dtype: float64

In [284]:
# find agencies with the largest range of salaries
df['Salary Range'] = df['Salary Range To'] - df['Salary Range From']
df.groupby("Agency")['Salary Range'].mean().sort_values().tail(10)[::-1]

Agency
DISTRICT ATTORNEY RICHMOND COU    99808.000000
DEPARTMENT OF SANITATION          92479.280000
NYC FIRE PENSION FUND             90707.000000
DEPARTMENT OF BUSINESS SERV.      82505.500000
DEPT OF INFO TECH & TELECOMM      81912.075472
DEPARTMENT OF TRANSPORTATION      62526.250000
CIVILIAN COMPLAINT REVIEW BD      59884.500000
NYC EMPLOYEES RETIREMENT SYS      52422.800256
TAXI & LIMOUSINE COMMISSION       35955.173913
NYC HOUSING AUTHORITY             33264.017910
Name: Salary Range, dtype: float64

## Shape

In [129]:
# how many rows and columns?
df.shape

(1218, 30)

In [131]:
# remind ourselves of the look of the data
df.sample(3)

Job ID                         Agency Posting Type  # Of Positions  \
694   434073  DEPT OF YOUTH & COMM DEV SRVS     Internal               1   
178   424227     FINANCIAL INFO SVCS AGENCY     Internal               1   
1025  444520    DEPARTMENT OF INVESTIGATION     Internal               1   

                    Business Title             Civil Service Title  \
694   Cornerstone Program Director  ADMIN COMMUNITY RELATIONS SPEC   
178      Web Application Developer             SENIOR IT ARCHITECT   
1025          Director of Outreach            SPECIAL INVESTIGATOR   

     Title Classification Title Code No Level  \
694         Competitive-1         1002F    00   
178     Non-Competitive-5         95711    00   
1025             Exempt-4         31130    00   

                                           Job Category  ...  \
694           Constituent Services & Community Programs  ...   
178                       Technology, Data & Innovation  ...   
1025  Communications & Intergovernmental Affairs Pol...  ...   

                                 Additional Information  \
694   This position is open to qualified persons wit...   
178                                                P293   
1025                                                NaN   

                                               To Apply  \
694   Search for the Job ID # 434073 External Candid...   
178   External applicants please visit https://a127-...   
1025  All current City Employees may apply by going ...   

                        Hours/Shift  Work Location 1 Recruitment Contact  \
694     35 Hours Per Week (minimum)              NaN                 NaN   
178   Monday - Friday, 9am  to 5pm.              NaN                 NaN   
1025                            NaN  180 Maiden Lane                 NaN   

                                  Residency Requirement Posting Date  \
694   New York City residency is generally required ...   02/21/2020   
178   New York City Residency is not required for th...   11/27/2019   
1025  New York City residency is generally required ...   09/14/2020   

       Post Until Posting Updated Process Date  
694           NaN      02/21/2020   04/13/2021  
178           NaN      11/27/2019   04/13/2021  
1025  30-JUN-2021      03/10/2021   04/13/2021  

[3 rows x 30 columns]

In [132]:
# flip the dataframe so columns become rows and rows become columns
df.transpose().head()

0                               1     \
Job ID                         457043                          231945   
Agency          NYC HOUSING AUTHORITY       ADMIN FOR CHILDREN'S SVCS   
Posting Type                 Internal                        Internal   
# Of Positions                      1                               9   
Business Title  Chief Privacy Officer  Quality Improvement Specialist   

                                             2     \
Job ID                                     456223   
Agency               DEPT OF INFO TECH & TELECOMM   
Posting Type                             Internal   
# Of Positions                                  2   
Business Title  System Access Management Engineer   

                                                       3     \
Job ID                                               460503   
Agency                       DEPT OF ENVIRONMENT PROTECTION   
Posting Type                                       Internal   
# Of Positions                                            1   
Business Title  2021-BWT-001-Construction Management Intern   

                                  4     \
Job ID                          293033   
Agency           NYC HOUSING AUTHORITY   
Posting Type                  External   
# Of Positions                       1   
Business Title  Deputy Press Secretary   

                                                             5     \
Job ID                                                     451777   
Agency                                  OFFICE OF THE COMPTROLLER   
Posting Type                                             External   
# Of Positions                                                  1   
Business Title  Director - Diversity & Inclusion and Emerging ...   

                                          6                           7     \
Job ID                                  458663                      458506   
Agency          PRESIDENT BOROUGH OF MANHATTAN  DEPT OF PARKS & RECREATION   
Posting Type                          Internal                    Internal   
# Of Positions                               1                          60   
Business Title  Information Technology Manager          City Seasonal Aide   

                                      8                            9     ...  \
Job ID                              458506                       460791  ...   
Agency          DEPT OF PARKS & RECREATION   FINANCIAL INFO SVCS AGENCY  ...   
Posting Type                      Internal                     Internal  ...   
# Of Positions                          60                            1  ...   
Business Title          City Seasonal Aide  MAINFRAME SYSTEM PROGRAMMER  ...   

                                                     1208  \
Job ID                                             316149   
Agency                              NYC HOUSING AUTHORITY   
Posting Type                                     External   
# Of Positions                                          5   
Business Title  Construction Safety and Quality Inspector   

                                    1209                          1210  \
Job ID                            432393                        445338   
Agency          DEPARTMENT OF CORRECTION  DEPT OF INFO TECH & TELECOMM   
Posting Type                    External                      Internal   
# Of Positions                         5                             5   
Business Title              EXTERMINATOR   Cyber Senior Threat Anaylst   

                                                         1211  \
Job ID                                                 395999   
Agency                                         LAW DEPARTMENT   
Posting Type                                         Internal   
# Of Positions                                              1   
Business Title  Certified IT Developer (Applications) Level 3   

                                                         1212  \
Job ID    